In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bitextsyntheticdata/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv


In [2]:
bitext = pd.read_csv('/kaggle/input/bitextsyntheticdata/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv')

In [3]:
bitext = bitext[bitext['intent']=='cancel_order']
bitext

,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...
...,...,...,...,...,...
993,BL,I need help to cancel purchase {{Order Number}},ORDER,cancel_order,I'm fully aware that you're seeking assistance...
994,BL,I want help to cancel purchase {{Order Number}},ORDER,cancel_order,You've come to the right place! I understand t...
995,BIL,how to cancel purchase {{Order Number}}?,ORDER,cancel_order,I've decoded that you're looking for assistanc...
996,BLZ,I want help cancellkng purchase {{Order Number}},ORDER,cancel_order,I realized that you're seeking assistance with...


In [4]:
!pip install -U -q "google-generativeai>=0.8.3" chromadb
!pip install --upgrade gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires kubernetes<27,>=8.0.0, but you have kubernetes 31.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 26.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 92.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing installation: tomlkit 0.13.2
    Uninstalling tomlkit-0.13.2:
      Successfully uninstalled tomlkit-0.13.2
  Attempting uninstall: python-multipar

In [6]:
import google.generativeai as genai
from IPython.display import Markdown

In [7]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [8]:
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)

models/embedding-001
models/text-embedding-004


In [9]:
Question1 = "Question: i have a question about cancelling order {{Order Number}} Answer: I've been informed that you have a question about canceling order {{Order Number}}. I'm here to assist you! Please go ahead and let me know what specific question you have, and I'll provide you with all the information and guidance you need. Your satisfaction is my top priority."
Question2 = '''Question:  I need to cancel order {{Order Number}} Answer: I can sense that you're seeking assistance with canceling your purchase with the purchase number {{Order Number}}. I apologize for any inconvenience caused, and I'm here to guide you through the process.

To cancel your purchase, please follow these steps:

1. Log into your account: Visit our {{Online Company Portal Info}} and sign in using your credentials.
2. Locate your order: Once logged in, navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section to find the purchase with the order number {{Order Number}}.
3. Initiate the cancellation: Upon finding your purchase, look for the option labeled '{{Online Order Interaction}}' and click on it to initiate the cancellation process.
4. Confirm the cancellation: Our system might ask for confirmation or feedback regarding the cancellation. Kindly provide the necessary information to complete the process.
5. Seek additional support: If you encounter any difficulties or require further assistance, our dedicated customer support team is available during {{Customer Support Hours}}. You can contact us at {{Customer Support Phone Number}} or use the Live Chat feature on our {{Website URL}}.

We appreciate your understanding and value your satisfaction. Should you have any more questions or concerns, feel free to reach out to us. We're here to help you.
'''
Question3 = ''' Question: I have a question regarding the cancellation process {{Order Number}} Answer: I've understood you have a question regarding canceling order {{Order Number}}, and I'm here to provide you with the information you need. Please go ahead and ask your question, and I'll do my best to assist you. '''
Question4 = '''Question: i dont know how to add some products to order {{Order Number}} Answer: "We understand that you're unsure about how to add products to order number #{{Order Number}}. We're here to help you with that. To add products to your order, please follow these steps:

1. Log in to your account on our website.
2. Go to the ""My Orders"" section.
3. Locate the order with number #{{Order Number}}.
4. Click on the ""Edit Order"" or ""Add Items"" button.
5. Browse our product catalog and select the items you wish to add.
6. Adjust the quantity if needed.
7. Click on the ""Add to Order"" button.
8. Review your updated order to ensure the changes are correct.
9. Proceed to the checkout page to complete your purchase.

If you encounter any difficulties or have further questions, feel free to reach out to our customer support team. They are available {{Customer Support Hours}} at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website URL}}. We're here to assist you every step of the way to ensure a smooth and satisfying shopping experience."'''

Question5 = '''Question: can uhelp me correct purchase {{Order Number}} {{Order Number}}. Answer: Happy to get your message to us for help with correcting purchase {{Order Number}}. I truly understand the importance of ensuring that your purchase is accurate and meets your expectations. To assist you in making the necessary corrections, could you please provide more details about the specific changes you would like to make to your purchase? This will enable me to better guide you through the process and ensure that everything is resolved to your satisfaction.'''
Question6 ='''Question: how do i edit purchase {{Order Number}}. Answer: We understand that you would like to edit your purchase with the order number {{Order Number}}. To assist you with this, please follow these steps:

1. Log in to your account on our website or app.
2. Navigate to the ""My Orders"" or ""Order History"" section.
3. Locate the purchase with the order number {{Order Number}}.
4. Click on the ""Edit"" or ""Modify"" option next to the purchase.
5. Review the available editable fields such as product quantity, color, size, or any other applicable parameters.
6. Make the necessary changes to the purchase details as per your requirements.
7. Double-check the modifications you made to ensure accuracy.
8. Once you are satisfied with the changes, click on the ""Save"" or ""Update"" button.
9. Your purchase with the order number {{Order Number}} will now be updated with the edits you made.

If you encounter any difficulties or require further assistance during this process, please feel free to let us know. We're here to help!"""'''
Question7= '''Question: what do I have to do to change an item of purchase {{Order Number}}? Answer:I realized that you need to change an item in your purchase with the number {{Order Number}}. To proceed with the modification, you can follow these steps:

1. Go to your account dashboard on our website or app.
2. Navigate to the "My Orders" or "Order History" section.
3. Locate the purchase with the number {{Order Number}}.
4. Click on the "Modify" or "Edit" option next to the item you wish to change.
5. Follow the prompts to select a new item or make the desired modifications.
6. Review the changes and confirm the update.

If you encounter any difficulties or need further assistance during this process, please don't hesitate to reach out to our customer support team. We're here to help and ensure a smooth modification experience for you.'''
Question8= '''Question: I need help swapping an item of order {{Order Number}} Answer: Thank you for bringing this to our attention. We understand that you would like to swap an item in your order with order number {{Order Number}}. We're here to assist you with this request. To proceed with the swap, we kindly ask you to provide us with the details of the item you would like to exchange and the replacement item you would like to receive. This way, we can ensure a smooth and seamless process for you. We appreciate your patience and cooperation as we work to resolve this matter to your satisfaction.'''
Questions = [Question1, Question2,Question3, Question4, Question5, Question6, Question7, Question8]

In [10]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        response = genai.embed_content(
            model="models/text-embedding-004",
            content=input,
            task_type=embedding_task,
            request_options=retry_policy,
        )
        return response["embedding"]

In [11]:
import chromadb

DB_NAME = "customerservicenewonedb"
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=Questions, ids=[str(i) for i in range(len(Questions))])

In [12]:
db.count()

8

In [13]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")

In [14]:
def generateResponse(instruction):
    embed_fn.document_mode = False

    # Search the Chroma DB using the specified query.
    query = instruction

    result = db.query(query_texts=[query], n_results=1)
    [[passage]] = result["documents"]

    #Markdown(passage)

    passage_oneline = passage.replace("\n", " ")
    query_oneline = query.replace("\n", " ")
    
      # This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
    prompt = f"""You are a customer service chat bot that answers questions using text from the reference passage included below. 
    Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
    However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
    strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it and inform user that they don't know how to answer this. 
    Make sure that you are empathetic to the concerns of the customer, and give them space to guide the conversation.

    QUESTION: {query_oneline}
    PASSAGE: {passage_oneline}
    """
    #print(prompt)
    print('I am about to generate content')
    generation_config = genai.GenerationConfig(temperature=0.7)  
    ans = model.generate_content(prompt, generation_config=generation_config)
    return ans.text

In [15]:
import gradio as gr

def greet(instruction):
    return generateResponse(instruction)

demo = gr.Interface(
    fn=greet,
    inputs=["text"],
    outputs=["text"],
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://a35c5dd0c354a582d4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


I am about to generate content
I am about to generate content


In [ ]:
# # Switch to query mode when generating embeddings.
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# for index, item in bitext.iterrows():
#     print('I am checking ',index)
#     if index>100: 
#         break
#     embed_fn.document_mode = False

#     # Search the Chroma DB using the specified query.
#     query = item['instruction']

#     result = db.query(query_texts=[query], n_results=1)
#     [[passage]] = result["documents"]

#     #Markdown(passage)

#     passage_oneline = passage.replace("\n", " ")
#     query_oneline = query.replace("\n", " ")

#     # This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
#     prompt = f"""You are a customer service chat bot that answers questions using text from the reference passage included below. 
#     Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
#     However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
#     strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it. 
#     Make sure that you are empathetic to the concerns of the customer, and give them space to guide the conversation.

#     QUESTION: {query_oneline}
#     PASSAGE: {passage_oneline}
#     """
#     #print(prompt)
#     print('I am about to generate content')
#     answer = model.generate_content(prompt)
#     #Markdown(answer.text)
#     passages = [answer.text, item['response']]
#     embeddings = []
#     for passage in passages:
#         print('Embedding passages')
#         response = genai.embed_content(
#             model='models/text-embedding-004',
#             content=passage,
#             task_type='semantic_similarity'
#         )
#         embeddings.append(response["embedding"])



#     similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])
#     print(f"Cosine Similarity: {similarity_score[0][0]}")

In [ ]:
# # Switch to query mode when generating embeddings.
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np
# questions = ['i have a question about cancelling oorder {{Order Number}}', 
#              'problem with canceling purchase {{Order Number}}',  
#              'i have problems with canceling order {{Order Number}}']
# answers = [""" I've been informed that you have a question about canceling order {{Order Number}}. I'm here to assist you! Please go ahead and let me know what specific question you have, and I'll provide you with all the information and guidance you need. Your satisfaction is my top priority.""", 
#           """I'm aware that you're facing difficulties canceling your purchase with the order number {{Order Number}}. I apologize for any inconvenience this may have caused you. Our team is committed to assisting you in resolving this issue. Here are a few steps you can take to help resolve the problem:

# 1. Double-check the Order Details: Make sure you have entered the correct order number. It's important to ensure accuracy when canceling purchases.
# 2. Try Different Channels: If you're encountering issues canceling the purchase online, you can reach out to our customer support team through various channels such as phone, email, or live chat. They will be able to guide you through the cancellation process.
# 3. Provide Additional Information: Sometimes, certain purchases require additional details for cancellation. Our customer support team can provide you with the necessary instructions and requested information to facilitate the cancellation process.
# 4. Be Patient: Cancelling a purchase can sometimes take a little longer due to system updates or verification processes. Rest assured, our team is working diligently to resolve the issue as quickly as possible.

# If you need further assistance, please feel free to provide any additional information or contact our customer support team during {{Customer Support Hours}} at {{Customer Support Phone Number}}. We appreciate your patience and understanding as we work to resolve this matter.""",
# """ I've been informed that you're facing some difficulties with canceling order {{Order Number}}, and I sincerely apologize for the inconvenience caused. Please allow me to assist you. 

# To resolve this issue, I recommend trying the following steps:

# 1. Double-check the Order Number: Ensure that you have entered the correct order number {{Order Number}} for cancellation.

# 2. Contact Customer Support: If you're still encountering problems, I highly recommend reaching out to our dedicated customer support team during {{Customer Support Hours}}. They are well-equipped to address any technical glitches or provide guidance on canceling your order.

# 3. Provide Details: When contacting our customer support team, kindly provide them with the specific details of your issue, such as the order number and any error messages you've encountered. This will help them quickly identify the problem and offer tailored solutions.

# Rest assured that we are committed to resolving this matter promptly and ensuring your satisfaction. Thank you for your patience and understanding.
# """          ]
# index = 0
# for question, answer in zip(questions, answers):
#     print('I am checking ',index)
#     if index>100: 
#         break
#     embed_fn.document_mode = False

#     # Search the Chroma DB using the specified query.
#     query = question

#     result = db.query(query_texts=[query], n_results=1)
#     [[passage]] = result["documents"]

#     #Markdown(passage)

#     passage_oneline = passage.replace("\n", " ")
#     query_oneline = query.replace("\n", " ")

#     # This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
#     prompt = f"""You are a customer service chat bot that answers questions using text from the reference passage included below. 
#     Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
#     However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
#     strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it. 
#     Make sure that you are empathetic to the concerns of the customer, and give them space to guide the conversation.

#     QUESTION: {query_oneline}
#     PASSAGE: {passage_oneline}
#     """
#     #print(prompt)
#     print('I am about to generate content')
#     generation_config = genai.GenerationConfig(temperature=0.7)  
#     ans = model.generate_content(prompt, generation_config=generation_config)
#     #Markdown(answer.text)
#     print(ans.text)
#     passages = [ans.text, answer]
#     embeddings = []
#     for passage in passages:
#         print('Embedding passages')
#         response = genai.embed_content(
#             model='models/text-embedding-004',
#             content=passage,
#             task_type='semantic_similarity'
#         )
#         embeddings.append(response["embedding"])



#     similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])
#     print(f"Cosine Similarity: {similarity_score[0][0]}")

I am checking  0
I am about to generate content
I understand you'd like to cancel order {{Order Number}}, and I'm sorry for any trouble this may be causing.  To cancel, first go to our website at {{Online Company Portal Info}} and log in.  Once you're logged in, look for the "{{Online Order Interaction}}" section (it might also be listed as "{{Online Order Interaction}}") to find your order, {{Order Number}}.  Then, find the option that says "{{Online Order Interaction}}" and click it to start the cancellation. You might need to confirm the cancellation with some extra information.  If you run into any problems at all, our customer support team is here to help during {{Customer Support Hours}}. You can reach them at {{Customer Support Phone Number}} or through the Live Chat on {{Website URL}}.  Please let me know if you have any other questions!

Embedding passages
Embedding passages
Cosine Similarity: 0.8125799512164464
I am checking  0
I am about to generate content


In [ ]:
# passage_oneline = passage.replace("\n", " ")
# query_oneline = query.replace("\n", " ")

# # This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
# prompt = f"""You are a customer service chat bot that answers questions using text from the reference passage included below. 
# Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
# However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
# strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it. 
# Make sure that you are empathetic to the concerns of the customer, and give them space to guide the conversation.

# QUESTION: {query_oneline}
# PASSAGE: {passage_oneline}
# """
# print(prompt)

In [ ]:
# passages = [
#        """ I'm sorry you're having trouble cancelling your purchase! I understand that can be frustrating. To help you out, let's go through the steps together. First, you'll need to log into your account on our website at {{Online Company Portal Info}} using your username and password. Once you're logged in, look for the "{{Online Order Interaction}}" section (it might also be listed as "{{Online Order Interaction}}") to find the order you want to cancel. The order number is {{Order Number}}. After you locate it, you should see an option that says "{{Online Order Interaction}}" – click on that to start the cancellation. You might need to confirm the cancellation with some additional information. If you run into any snags at all, or if anything is unclear, our customer support team is available during {{Customer Support Hours}}. You can reach them at {{Customer Support Phone Number}} or through the Live Chat on our website, {{Website URL}}. We're here to help you through this process. Is there anything else I can assist you with today?""",
#        """I can sense that you're seeking assistance with canceling your purchase with the purchase number {{Order Number}}. I apologize for any inconvenience caused, and I'm here to guide you through the process.

# To cancel your purchase, please follow these steps:

# 1. Log into your account: Visit our {{Online Company Portal Info}} and sign in using your credentials.
# 2. Locate your order: Once logged in, navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section to find the purchase with the order number {{Order Number}}.
# 3. Initiate the cancellation: Upon finding your purchase, look for the option labeled '{{Online Order Interaction}}' and click on it to initiate the cancellation process.
# 4. Confirm the cancellation: Our system might ask for confirmation or feedback regarding the cancellation. Kindly provide the necessary information to complete the process.
# 5. Seek additional support: If you encounter any difficulties or require further assistance, our dedicated customer support team is available during {{Customer Support Hours}}. You can contact us at {{Customer Support Phone Number}} or use the Live Chat feature on our {{Website URL}}.

# We appreciate your understanding and value your satisfaction. Should you have any more questions or concerns, feel free to reach out to us. We're here to help you.
# """

#    ]

In [ ]:
# embeddings = []
# for passage in passages:
#     response = genai.embed_content(
#         model='models/text-embedding-004',
#         content=passage,
#         task_type='semantic_similarity'
#     )
#     embeddings.append(response["embedding"])

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])
# print(f"Cosine Similarity: {similarity_score[0][0]}")